In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 30
S_LAYER_SIZE = 10
LATENT_SIZE = 3
C = 0.5

In [3]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.set_printoptions(linewidth=140)

torch.Size([50000, 11])


In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.l4 = nn.Linear(S_LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        x = F.relu(self.l1(x))

        # return the mu and the sigma
        return self.l2a(x), self.l2b(x)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        x = F.relu(self.l3(x))

        return torch.sigmoid(self.l4(x)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + C*KLD

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
train_data=train_data[torch.randperm(train_data.size()[0])]
test_data=test_data[torch.randperm(test_data.size()[0])]

In [10]:
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

In [11]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

Epoch: 0, Loss: 7.23754759399414
Epoch: 1, Loss: 6.9105022802734375
Epoch: 2, Loss: 6.822154013671875
Epoch: 3, Loss: 6.7828437561035155
Epoch: 4, Loss: 6.766266428222656
Epoch: 5, Loss: 6.757202491455078
Epoch: 6, Loss: 6.754297359619141
Epoch: 7, Loss: 6.750685964355469
Epoch: 8, Loss: 6.7469140234375
Epoch: 9, Loss: 6.748092197265625
Epoch: 10, Loss: 6.745354976806641
Epoch: 11, Loss: 6.748027515869141
Epoch: 12, Loss: 6.74300748046875
Epoch: 13, Loss: 6.745528076171875
Epoch: 14, Loss: 6.744858355712891
Epoch: 15, Loss: 6.7433385144042965
Epoch: 16, Loss: 6.743844017333984
Epoch: 17, Loss: 6.746955979003906
Epoch: 18, Loss: 6.745454392089844
Epoch: 19, Loss: 6.741417415771484
Epoch: 20, Loss: 6.743091281738281
Epoch: 21, Loss: 6.743754403076172
Epoch: 22, Loss: 6.74439776977539
Epoch: 23, Loss: 6.7446217333984375
Epoch: 24, Loss: 6.747117987060546
Epoch: 25, Loss: 6.743473489990234
Epoch: 26, Loss: 6.745346912841796
Epoch: 27, Loss: 6.744711076660156
Epoch: 28, Loss: 6.745174241943

In [12]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [13]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [14]:
from torch.autograd import Variable

print("Action")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 1).to("cuda")
    print(sample[0][5])
    x = vae1(sample)
    print(x[0][0][5], '\n')

Action
tensor(0.1193, device='cuda:0')
tensor(0.4997, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.7109, device='cuda:0')
tensor(0.4995, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0246, device='cuda:0')
tensor(0.4996, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1223, device='cuda:0')
tensor(0.4997, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.9242, device='cuda:0')
tensor(0.4998, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4520, device='cuda:0')
tensor(0.4995, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8199, device='cuda:0')
tensor(0.4995, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0477, device='cuda:0')
tensor(0.4992, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.7109, device='cuda:0')
tensor(0.4997, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.5853, device='cuda:0')
tensor(0.4992, device='cuda:0', grad_fn=<SelectBackward>) 



In [15]:
print("Reward")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 0.15).to("cuda")
    print(sample[0][9])
    x = vae1(sample)
    print(x[0][0][9], '\n')

Reward
tensor(0.0087, device='cuda:0')
tensor(0.0500, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0366, device='cuda:0')
tensor(0.0501, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0759, device='cuda:0')
tensor(0.0510, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1008, device='cuda:0')
tensor(0.0504, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0494, device='cuda:0')
tensor(0.0498, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0874, device='cuda:0')
tensor(0.0491, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1323, device='cuda:0')
tensor(0.0498, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0228, device='cuda:0')
tensor(0.0498, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0919, device='cuda:0')
tensor(0.0501, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0689, device='cuda:0')
tensor(0.0500, device='cuda:0', grad_fn=<SelectBackward>) 



In [16]:
torch.set_printoptions(precision=3, sci_mode=False, linewidth=140, profile=None)
import random
for i in range(5):
    k = random.randint(1, 400)
    sample = test_data[k].to(device)
    x, m, s = vae1(sample)
    loss = loss_function(x, sample, m, s)
    
    x = x.to("cpu")
    sample = sample.to("cpu")
    
    print(f"Original:       {sample}")
    sample = sample.to(device)
    print(f"Reconstruction: {x}\n")
    torch.set_printoptions(precision=3, sci_mode=False, linewidth=140, profile="short")
    print(f"Latent: {vae1.reparameterize(list(vae1.encode(sample))[0], list(vae1.encode(sample))[1])}")
    print(f"\nLoss: {loss}\n\n")
    

Original:       tensor([0.500, 0.500, 0.500, 0.500, 0.298, 0.500, 0.501, 0.477, 0.552, 0.050, 0.000])
Reconstruction: tensor([    0.500,     0.500,     0.497,     0.500,     0.493,     0.500,     0.499,     0.500,     0.498,     0.051,     0.000],
       grad_fn=<CopyBackwards>)

Latent: tensor([ 1.188, -1.156,  2.606], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 6.5112762451171875


Original:       tensor([0.500, 0.500, 0.500, 0.500, 0.729, 0.500, 0.499, 0.526, 0.441, 0.050, 0.000])
Reconstruction: tensor([    0.500,     0.499,     0.496,     0.501,     0.488,     0.500,     0.499,     0.499,     0.500,     0.050,     0.000],
       grad_fn=<CopyBackwards>)

Latent: tensor([0.284, 0.809, 0.779], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 6.528227806091309


Original:       tensor([0.501, 0.497, 0.551, 0.381, 0.989, 0.505, 0.490, 0.604, 0.263, 0.050, 0.000])
Reconstruction: tensor([0.500, 0.500, 0.498, 0.501, 0.498, 0.500, 0.500, 0.500, 0.500, 0.050, 0.003], grad_fn=<CopyBackw

In [17]:
sample = Variable(torch.rand(1, 3))

In [20]:
latents = []

for i in range(300):
    k = random.randint(1, 6000)
    sample = test_data[k].to(device)
    l = vae1.reparameterize(list(vae1.encode(sample))[0], list(vae1.encode(sample))[1])
    latents.append([i.item() for i in l])
    
np.save(f"Latents/latent{C}", latents)
    

In [19]:
res = [0, 0, 0]
k = []
for i in range(10000):
    sample = Variable(torch.randn(1, LATENT_SIZE))
    a = vae1.decode(sample.to("cuda"))[0]
    num = float(sum(list(a[10:])))
    k.append(num)
    if num < 0.5:
        res[0] += 1
    elif num <0.6:
        res[1] += 1
    else:
        res[2] += 1
        
d = 0
nd = 0

for a in test_data:
    if a[10] == 1:
        d += 1
    else:
        nd += 1
        
        
print(f"UNBALANCED DATASET")
print(f"counts of intervals: [0, 0.1]: {res[0]}, [0.1, 0.7]: {res[1]}, [0.7, 1]: {res[2]}")
print(f"-- so the ratio to done and not done (deleting the middle ones) is {res[2]/(res[2]+res[0])}")
print(f"The real ration is {d/(nd+d)}")

UNBALANCED DATASET
counts of intervals: [0, 0.1]: 8599, [0.1, 0.7]: 106, [0.7, 1]: 1295
-- so the ratio to done and not done (deleting the middle ones) is 0.13088740650899536
The real ration is 0.17664
